<a href="https://colab.research.google.com/github/NicolaGabriele/powerline_Segmentation_project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [57]:
import torch
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import json
TRAIN_DIR = '/content/drive/MyDrive/trainingset'
TEST_DIR = '/content/drive/MyDrive/testset'
TRAIN_LABELS = '/content/drive/MyDrive/train.json'
TEST_LABELS =  '/content/drive/MyDrive/test.json'

In [61]:
class PowerLineDataset(torch.utils.data.Dataset):
  def __init__(self,img_dir,img_labels):
    self.img_dir = img_dir
    self.img_labels = json.load(open(img_labels))
    self.categories = [] #lista delle etichette di classe
    for c in self.img_labels['categories']:
      self.categories.append(c['name'])

  def __getitem__(self,idx):
    img_path = os.path.join(self.img_dir, self.img_labels['images'][idx]['file_name'])
    label = self.img_labels['annotations'][idx]
    image = torchvision.io.read_image(img_path)
    return image,label
  def __len__(self):
    return len(os.listdir(self.img_dir))

In [64]:
#caricamento dei dataset di train e di test
train = PowerLineDataset(TRAIN_DIR, TRAIN_LABELS)
test = PowerLineDataset(TEST_DIR, TEST_LABELS)
train[0]

(tensor([[[166, 177, 180,  ..., 186, 204, 174],
          [167, 170, 177,  ..., 173, 188, 167],
          [165, 167, 164,  ..., 167, 166, 173],
          ...,
          [169, 143, 144,  ...,  50,  54,  56],
          [165, 193, 170,  ...,  50,  52,  55],
          [176, 208, 196,  ...,  50,  52,  54]],
 
         [[134, 145, 147,  ..., 166, 182, 153],
          [135, 138, 144,  ..., 151, 166, 146],
          [133, 135, 131,  ..., 145, 142, 150],
          ...,
          [164, 136, 132,  ...,  45,  46,  48],
          [162, 186, 157,  ...,  45,  47,  50],
          [173, 203, 180,  ...,  45,  47,  49]],
 
         [[ 95, 107, 112,  ..., 116, 145, 122],
          [ 96, 100, 109,  ..., 102, 129, 115],
          [ 95,  97,  96,  ...,  98, 106, 119],
          ...,
          [ 70,  46,  50,  ...,  42,  44,  46],
          [ 67,  96,  78,  ...,  42,  44,  47],
          [ 78, 112, 103,  ...,  42,  44,  46]]], dtype=torch.uint8),
 {'segmentation': [[0.0,
    125.74074074074075,
    142.933238

In [63]:
#questo metodo è preso dai notebook e va adattato al caso specifico (loss ecc...)
criterion = torch.nn.CrossEntropyLoss()
learning_rate = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total_step = len(train)
batch_size = 64

num_epochs = 3

train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train) for i in range(num_epochs + 1)]

def train(epoch,model,criterion,optimizer,reshape=True):
    for batch_idx, (images, labels) in enumerate(train):
        # Move tensors to the configured device
        if reshape:
            images = images.reshape(-1, 28*28)
        #images = images.to(device)
        #labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_idx+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch, num_epochs, batch_idx+1, total_step, loss.item()))

        train_losses.append(loss.item())
        train_counter.append(
        (batch_idx*batch_size) + ((epoch-1)*len(train)))